In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [3]:
CUST=['R','H','M']
PROD= ['S','M','L','P']
PLANT= [1, 2, 3] 

In [20]:
AvailMaterial= 3500
Capacity= 1500
AvailLaborHour= {1:6000, 2:5000, 3:3000}
AvailMachineHour= {1:10000, 2:12500, 3:6000}

## Sale Prices

In [21]:
MaxProdSales={
    ('S','R'):300,
    ('S','H'):400,
    ('S','M'):300,
    ('M','R'):300,
    ('M','H'):300,
    ('M','M'):400,
    ('L','R'):500,
    ('L','H'):200,
    ('L','M'):300,
    ('P','R'):200,
    ('P','H'):400,
    ('P','M'):300
}

SalePrice={
    ('S','R'):17,
    ('S','H'):16,
    ('S','M'):16,
    ('M','R'):18,
    ('M','H'):18,
    ('M','M'):17,
    ('L','R'):22,
    ('L','H'):22,
    ('L','M'):23,
    ('P','R'):29,
    ('P','H'):26,
    ('P','M'):27
}

## Production and Shipping Cost

In [22]:
ShipCost={
    ('R',1):1.0,
    ('R',2):1.2,
    ('R',3):1.4,
    ('H',1):1.6,
    ('H',2):1.5,
    ('H',3):1.5,
    ('M',1):1.1,
    ('M',2):1.0,
    ('M',3):1.3
}

ProdCost={
    ('S',1):14,
    ('S',2):13,
    ('S',3):14,
    ('M',1):16,
    ('M',2):17,
    ('M',3):15,
    ('L',1):18,
    ('L',2):20,
    ('L',3):19,
    ('P',1):26,
    ('P',2):24,
    ('P',3):23
}

## Available resources

In [23]:
LaborHour={
    ('S',1):3,
    ('S',2):3.5,
    ('S',3):3,
    ('M',1):3,
    ('M',2):3.5,
    ('M',3):3.5,
    ('L',1):4,
    ('L',2):4.5,
    ('L',3):4,
    ('P',1):4,
    ('P',2):4.5,
    ('P',3):4.5
}

MachineHour={
    ('S',1):8,
    ('S',2):7,
    ('S',3):7.5,
    ('M',1):8.5,
    ('M',2):7,
    ('M',3):7.5,
    ('L',1):9,
    ('L',2):8,
    ('L',3):8.5,
    ('P',1):9,
    ('P',2):9,
    ('P',3):8.5
}

Material={
    ('S',1):1.0,
    ('S',2):1.1,
    ('S',3):1.1,
    ('M',1):1.1,
    ('M',2):1.0,
    ('M',3):1.1,
    ('L',1):1.2,
    ('L',2):1.1,
    ('L',3):1.3,
    ('P',1):1.3,
    ('P',2):1.4,
    ('P',3):1.3
}

In [24]:
Material

{('S', 1): 1.0,
 ('S', 2): 1.1,
 ('S', 3): 1.1,
 ('M', 1): 1.1,
 ('M', 2): 1.0,
 ('M', 3): 1.1,
 ('L', 1): 1.2,
 ('L', 2): 1.1,
 ('L', 3): 1.3,
 ('P', 1): 1.3,
 ('P', 2): 1.4,
 ('P', 3): 1.3}

In [25]:
# var ProduceP1{PROD, CUST} >= 0;
# var ProduceP2{PROD, CUST} >= 0;
# var ProduceP3{PROD, CUST} >= 0;

# Model Deployment

In [26]:
model = gp.Model('Total Revenue')

ProduceP1=model.addVars(PROD, CUST, name="P1")
ProduceP2=model.addVars(PROD, CUST, name="P2")
ProduceP3=model.addVars(PROD, CUST, name="P3")

### Labor Required

In [27]:
Labor_Required=model.addConstrs((gp.quicksum(LaborHour[i,p]*ProduceP2[i,j] for i in PROD for j in CUST) 
                                   <= AvailMachineHour[p] for p in PLANT), "Lab Req")

### Machine Required

In [28]:
Machine_Required=model.addConstrs((gp.quicksum(MachineHour[i,p]*ProduceP1[i,j] for i in PROD for j in CUST) 
                                   <= AvailMachineHour[p] for p in PLANT),"Mac Req")

### Material Required

In [29]:
Material_Req=model.addConstr((gp.quicksum(((ProduceP1[i,'H']+ProduceP1[i,'R']+ProduceP1[i,'M'])*Material[i,1]+
                        (ProduceP2[i,'H']+ProduceP2[i,'R']+ProduceP2[i,'M'])*Material[i,2]+
                        (ProduceP3[i,'H']+ProduceP3[i,'R']+ProduceP3[i,'M'])*Material[i,3]) for i in PROD)
                         <= AvailMaterial),"Mat Req")

### Max Sales

In [30]:
Max_Sales=model.addConstrs((SalePrice[i,j]*(ProduceP1[i,j]+ProduceP2[i,j]+ProduceP3[i,j])
                             <= MaxProdSales[i,j] for i in PROD for j in CUST),"Sls Lim")

### Inspection Capacity

In [31]:
Inspection=model.addConstr((gp.quicksum((ProduceP1[i,j]+ProduceP2[i,j]) 
                                          for i in PROD for j in CUST if j!='M' ) <= Capacity),"Inspection")

In [32]:
model.setObjective(
                    gp.quicksum(SalePrice[i,j]*(ProduceP1[i,j]+ProduceP2[i,j]+ProduceP3[i,j]) for i in PROD for j in CUST)
                    - gp.quicksum(((ProduceP1[i,'H']+ProduceP1[i,'R']+ProduceP1[i,'M'])*ProdCost[i,1]+
                       (ProduceP2[i,'H']+ProduceP2[i,'R']+ProduceP2[i,'M'])*ProdCost[i,2]+
                       (ProduceP3[i,'H']+ProduceP3[i,'R']+ProduceP3[i,'M'])*ProdCost[i,3]) for i in PROD)
                    -gp.quicksum((ProduceP1[i,j]*ShipCost[j,1]+ProduceP2[i,j]*ShipCost[j,2]+ProduceP3[i,j]*ShipCost[j,3])
                                 for i in PROD for j in CUST)
                    , GRB.MAXIMIZE)
    

In [33]:
# Verify model formulation

model.write('Case4.lp')

# Run optimization engine

model.optimize()


Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20 rows, 36 columns and 160 nonzeros
Model fingerprint: 0x10aa163b
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e-01, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+04]
Presolve removed 20 rows and 36 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1821965e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  4.182196458e+02


In [34]:
print(model.objVal)

418.2196458331921


In [24]:
# model.getConstr("Labor Required")

In [25]:
# model.getVars()

In [26]:
# X: Value in the current solution.
# obj: Linear objective coefficient.
# SAObjLow: Objective coefficient sensitivity information. 

print('Sensitivity Analysis (SA)\nObjVal =', model.ObjVal)
model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])

Sensitivity Analysis (SA)
ObjVal = 389.24905759789795

    Variable            X          Obj     SAObjLow      SAObjUp 
----------------------------------------------------------------
     P1[S,R]            0            2         -inf          2.8 
     P1[S,H]            0          0.4         -inf          1.5 
     P1[S,M]            0          0.9         -inf            2 
     P1[M,R]            0            1         -inf          1.6 
     P1[M,H]            0          0.4         -inf          1.5 
     P1[M,M]            0         -0.1         -inf          0.7 
     P1[L,R]      22.7273            3          1.6          inf 
     P1[L,H]      9.09091          2.4          1.5          inf 
     P1[L,M]      13.0435          3.9          2.7          inf 
     P1[P,R]            0            2         -inf          4.6 
     P1[P,H]            0         -1.6         -inf          1.5 
     P1[P,M]            0         -0.1         -inf          2.7 
     P2[S,R]      11.7

In [27]:
model.printAttr(['X', 'RC', 'LB', 'SALBLow', 'SALBUp', 'UB', 'SAUBLow', 'SAUBUp'])


    Variable            X           RC           LB      SALBLow       SALBUp           UB      SAUBLow       SAUBUp 
--------------------------------------------------------------------------------------------------------------------
     P1[S,R]            0         -0.8            0     -1950.74      11.7647          inf            0          inf 
     P1[S,H]            0         -1.1            0     -1950.74           25          inf            0          inf 
     P1[S,M]            0         -1.1            0     -1950.74         12.5          inf            0          inf 
     P1[M,R]            0         -0.6            0         -inf      16.6667          inf            0          inf 
     P1[M,H]            0         -1.1            0         -inf      16.6667          inf            0          inf 
     P1[M,M]            0         -0.8            0         -inf      23.5294          inf            0          inf 
     P1[L,R]      22.7273            0            0     

In [28]:
model.printAttr(['Sense','Pi','Slack', 'SARHSLow', 'RHS', 'SARHSUp'])


  Constraint        Sense           Pi        Slack     SARHSLow          RHS      SARHSUp 
------------------------------------------------------------------------------------------
  Lab Req[1]            <            0      9852.21      147.794        10000          inf 
  Lab Req[2]            <            0      12327.6      172.426        12500          inf 
  Lab Req[3]            <            0      5852.21      147.794         6000          inf 
  Mac Req[1]            <            0      9596.25      403.755        10000          inf 
  Mac Req[2]            <            0      12141.1      358.893        12500          inf 
  Mac Req[3]            <            0      5618.68      381.324         6000          inf 
     Mat Req            <            0      3286.02      213.984         3500          inf 
Sls Lim[S,R]            <     0.164706            0 -2.84217e-14          200      24534.1 
Sls Lim[S,H]            <      0.09375            0            0          400   